## Project 2       Jungang (Gordon) Chen            UIN: 5280097929

## 1)  Load the text dataset and text preprocessing

In [1]:
import pickle
import numpy as np
import tensorflow as tf

In [5]:
input_sequence = pickle.load(open('DS_5_train_input', 'rb'))
output_sequence = pickle.load(open('DS_5_train_output', 'rb'))
print(output_sequence[50])

c d a e d a e e b g a f f g b f c g b d a k j k c e a i i l m a d h ed b e a e ef c e a d eg eh a g ee ei a e ej 


In [3]:
text_pairs = []
for line in range(len(input_sequence)):
    inputish  = input_sequence[line]
    outputish = "[start] " + output_sequence[line] + "[end]"
    text_pairs.append((inputish, outputish))

import random
print(random.choice(text_pairs))

('a j b f a f c d a k c e a f b d b g a f c g a d b f a k c g c d ', '[start] b f c d c e b d b g a f g h a k f i a f e j c g b f c g c d a k ed ee a d m ef a f l eg a j d k eh [end]')


In [4]:
import random
random.shuffle(text_pairs)
print(len(text_pairs))
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

5000


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import string
import re

strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

src_vocab_size = 50
tgt_vocab_size = 50
sequence_length = 100

source_vectorization = layers.TextVectorization(
    max_tokens=src_vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=tgt_vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length+1, 
    standardize=custom_standardization,
)
    # standardize=custom_standardization,
train_inputish_texts = [pair[0] for pair in train_pairs]
print(type(train_inputish_texts))
print(train_inputish_texts[0])
train_outputish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_inputish_texts)
target_vectorization.adapt(train_outputish_texts)
print(len(source_vectorization.get_vocabulary()))

print(len(target_vectorization.get_vocabulary()))

# save the text vectorization for future testing
pickle.dump({'config': source_vectorization.get_config(),
             'weights': source_vectorization.get_weights()}
            , open("source_tv_layer.pkl", "wb"))

pickle.dump({'config': target_vectorization.get_config(),
             'weights': target_vectorization.get_weights()}
            , open("target_tv_layer.pkl", "wb"))

<class 'list'>
a d c g a d a h a g a e c d a f a h b f c e a k b e a e a j a k a e c d b g b e b f a g a d b d b f b g b d b g b e 
13
38


In [46]:
batch_size = 16
def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)      # unzip the text pairs
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

print(train_ds)

<CacheDataset element_spec=({'english': TensorSpec(shape=(None, 100), dtype=tf.int64, name=None), 'spanish': TensorSpec(shape=(None, 100), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 100), dtype=tf.int64, name=None))>


In [47]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (16, 100)
inputs['spanish'].shape: (16, 100)
targets.shape: (16, 100)


## 2) Build a transformer model with a stack of 2 encoder and 3 decoder

In [48]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [49]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [50]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [51]:
embed_dim = 64
dense_dim = 512
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, src_vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, tgt_vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(tgt_vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [52]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint("artificial_text_translation.keras",
                                    save_best_only=True)
]
transformer.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=callbacks)

Epoch 1/100
219/219 [==============================] - 17s 42ms/step - loss: 1.3977 - accuracy: 0.3003 - val_loss: 0.9445 - val_accuracy: 0.4171
Epoch 2/100
219/219 [==============================] - 8s 36ms/step - loss: 0.9582 - accuracy: 0.4283 - val_loss: 0.8373 - val_accuracy: 0.4801
Epoch 3/100
219/219 [==============================] - 8s 36ms/step - loss: 0.8140 - accuracy: 0.4949 - val_loss: 0.7469 - val_accuracy: 0.5327
Epoch 4/100
219/219 [==============================] - 8s 36ms/step - loss: 0.7233 - accuracy: 0.5442 - val_loss: 0.6144 - val_accuracy: 0.5941
Epoch 5/100
219/219 [==============================] - 8s 37ms/step - loss: 0.6597 - accuracy: 0.5772 - val_loss: 0.5778 - val_accuracy: 0.6152
Epoch 6/100
219/219 [==============================] - 8s 36ms/step - loss: 0.6219 - accuracy: 0.6008 - val_loss: 0.5233 - val_accuracy: 0.6432
Epoch 7/100
219/219 [==============================] - 8s 37ms/step - loss: 0.5846 - accuracy: 0.6221 - val_loss: 0.5132 - val_accuracy

## 3) Model Accuracy by randomly selecting 50 true/prediction testing sentences and compare them token by token

In [53]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 100

def decode_sequence(input_sentence, model):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = model(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

def transform_format(true_output_sequence, pred_sequence):
    true_output_sequence = true_output_sequence.split()
    pred_sequence  = pred_sequence.split()
    true_output_sequence.remove('[start]')
    true_output_sequence.remove('[end]')
    pred_sequence.remove('[start]')
    pred_sequence.remove('[end]')
    return true_output_sequence, pred_sequence


test_eng_texts = [pair[0] for pair in test_pairs]
test_spa_texts = [pair[1] for pair in test_pairs]

count_total = 0
count_match =0 
for i in np.random.randint(len(test_eng_texts),size=50):
    input_sequence = test_eng_texts[i]
    true_output_sequence = test_spa_texts[i]    
    pred_sequence  = decode_sequence(input_sequence,transformer)
    print(input_sequence)
    print(true_output_sequence)
    print(pred_sequence)
    true_output_sequence, pred_sequence=transform_format(true_output_sequence, pred_sequence)
    count_total+=len(true_output_sequence)
    for i, token in enumerate(pred_sequence):
        if i<len(true_output_sequence):
            if token == true_output_sequence[i]:
              count_match+=1
        else:
            break
  # print(count_match)
  # print(len(true_output_sequence))
  # print(count_match/len(true_output_sequence))

print(count_match/count_total)

a g a k a d c e a d b f a k b e a j a g a f b d c e b f b e c d c g b e 
[start] c e b f b e b d c e a f g h b f a g i j b e c d a j k l m a k f ed a d e ee a d d ef c g a k eg eh b e a g ei ej [end]
[start] c e b f b e b d c e a f g h b f a g i j b e c d a j k l m a k f ed a d e ee a d d ef c g a k eg eh b e a g ei ej [end]
a h a d b g a d a i b d b d a k b e a e a e a h a k c e b d c e b d c e c d c e 
[start] b g b d b d b e c e b d a k h i c e b d a h j k l a e m a e ed a k g ee a i e f ef c e a d eg eh a d d ei c d c e a h ej ek el [end]
[start] b g b d b d b e c e b d a k h i c e b d a h j k l a e m a k g ed a i e f ee c e a d ef eg a d d eh c d c e a h ei ej ek [end]
a f a f a d a d a d c f b d b g a d b d c d c g a d a g a i a g c g c d c f a e c d b d b g 
[start] c f b d a d d e b g a d f g b d c d a d i j a d h k c g a f l m c g c d a g ee ef c f c d a e ei a i eg eh ej b d a g ek el b g a d em fd a f ed fe [end]
[start] c f b d a d d e b g a d f g b d c d a d i j a d h k c 

### as can be seen, the accuracy is 0.77 for 50 testing text sentences

## 4) Reload the model and test again

In [56]:
# At loading time, register the custom objects with a `custom_object_scope`:
saved_model = keras.models.load_model("artificial_text_translation.keras", custom_objects={
'TransformerEncoder': TransformerEncoder,
'TransformerDecoder': TransformerDecoder,
'PositionalEmbedding': PositionalEmbedding
})